# INTRODUCTION

In [ ]:

# Import SparkSession
import pyspark
from delta import configure_spark_with_delta_pip

builder = pyspark.sql.SparkSession.builder.appName("STREAMING_DWH") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark.sql("CREATE SCHEMA IF NOT EXISTS demo;")
spark.sql("USE SCHEMA demo;")


In [ ]:
# To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)

# Create the streaming_df to read from input directory
streaming_df = spark \
    .readStream \
    .format("json") \
    .option("maxFilesPerTrigger", 1) \
    .load("data/product/")

streaming_df.printSchema()

In [ ]:
from pyspark.sql.functions import current_timestamp, input_file_name

def bronze_streaming_ingestion(source, target):
    # Generates a source path based on table name, reads all files from that and inserts into bronze schema

    query = (
        spark.readStream
        .format("json")
        .option("maxFilesPerTrigger", 1)
        .load(source)
        .withColumn("meta_timestamp", current_timestamp())
        .withColumn("meta_filename", input_file_name())
        .writeStream
        .outputMode("append")
        .format("delta")
        .option("checkpointLocation", f"spark-warehouse/_checkpoints/{target}")
        .toTable(target)
    )
    return query

query1 = bronze_streaming_ingestion(source="data/inventory", target="bronze_inventory")
query2 = bronze_streaming_ingestion(source="data/product", target="bronze_product")
query3 = bronze_streaming_ingestion(source="data/purchase", target="bronze_purchase")

# Use the code 
spark.streams.awaitAnyTermination()


In [ ]:
spark.sql("SELECT * FROM bronze_product").show(15)

# SLOWLY CHANGING DIMENSIONS (SCD) - TYPE 2

In [ ]:

from pyspark.sql.functions import md5, concat_ws, lit, row_number, monotonically_increasing_id
from pyspark.sql.types import BooleanType, TimestampType
from pyspark.sql.window import Window

def silver_scd2_dim_table(source, target, merge_key, timestamp_key, surrogate_column_name):

    # Get list of all columns that doesn't contain "meta_"
    df = spark.sql(f"select * from {source}")
    df = df.withColumn("meta_hashdiff", md5(concat_ws("||", *[c for c in df.columns if "meta_" not in c])))
    df = df.withColumn("meta_is_current", lit(1).cast(BooleanType()))
    df = df.withColumn("meta_valid_from", df[timestamp_key])
    df = df.withColumn("meta_valid_to", lit('9999-12-31').cast(TimestampType()))

    # Add partition column
    window_spec  = Window.partitionBy(merge_key).orderBy(timestamp_key)
    df = df.withColumn("meta_sequence", row_number().over(window_spec))
    df = df.withColumn(surrogate_column_name, monotonically_increasing_id())

    # Create an empty Delta table with the same schema
    tmp_view_name = "temporaryView"
    df.createOrReplaceTempView(tmp_view_name)
    spark.sql(f"CREATE TABLE IF NOT EXISTS {target} LIKE {tmp_view_name} USING DELTA")

    # Get list of sequences
    lst_sequence = sorted([p.meta_sequence for p in df.select('meta_sequence').distinct().collect()])

    # Run SCD2 table 
    for seq_num in lst_sequence:
        merge_query = f"""
            MERGE INTO {target} AS target
            USING (
                SELECT * FROM {tmp_view_name}
                WHERE meta_sequence = {seq_num}
            ) AS source ON target.{merge_key} = source.{merge_key}
            WHEN MATCHED AND target.meta_is_current = true AND target.meta_hashdiff <> source.meta_hashdiff
                THEN UPDATE SET meta_is_current = false, meta_valid_to = source.{timestamp_key}
            WHEN NOT MATCHED 
                THEN INSERT *
        """

        # Perform merge query
        spark.sql(merge_query).show()

        insert_query = f"""
            INSERT INTO {target}
            SELECT * FROM 
            (
                SELECT source.* 
                FROM {tmp_view_name} source
                JOIN {target} target ON target.{merge_key} = source.{merge_key}
                WHERE source.meta_sequence = {seq_num}
                AND target.meta_hashdiff <> source.meta_hashdiff 
            )
        """
        
        # Perform insert query
        spark.sql(insert_query)

In [ ]:
# Create SCD2 tables
spark.sql("DROP TABLE IF EXISTS silver_product_scd2")

silver_scd2_dim_table(
    source = "bronze_product",
    target = "silver_product_scd2",
    merge_key = "product_id",
    timestamp_key = "event_time",
    surrogate_column_name = "product_sid"
)

In [ ]:
spark.sql("select * from silver_product_scd2 order by product_sid").show(10, truncate=False)

# Creating Silver Fact Streaming Tables

In [ ]:
from util_functions import generate_dim_table_references, reorder_columns_in_dataframe

def silver_fact_table(
    source : str, 
    target : str,
    surrogate_key : str,
    timestamp_key : str,
    dim_table_refs : dict
):

    # Generate and run SQL query
    df = spark.sql(generate_dim_table_references(source, timestamp_key, dim_table_refs))

    # Generate surrogate key
    df = df.withColumn(surrogate_key, monotonically_increasing_id())

    # Reorder columns
    df = reorder_columns_in_dataframe(
        df=df, 
        columns_to_front=[surrogate_key] + [row["surrogate_key"] for row in dim_table_refs],
        columns_to_delete=["meta_filename"]
    )

    df.show(5)


In [ ]:


silver_fact_table (
    source="bronze_purchase",
    target="silver_purchase",
    surrogate_key="transaction_sid",
    timestamp_key="transaction_time",
    dim_table_refs=[{"table_name": "silver_product_scd2", "merge_key": "product_id", "surrogate_key": "product_sid"}]
)

silver_fact_table (
    source="bronze_inventory",
    target="silver_inventory",
    surrogate_key="inventory_sid",
    timestamp_key="event_time",
    dim_table_refs=[{"table_name": "silver_product_scd2", "merge_key": "product_id", "surrogate_key": "product_sid"}]
)

# if spark.catalog.tableExists(target):
#     df = spark.sql(f"SELECT * FROM {source} WHERE {source}.{timestamp_key} > (SELECT MAX({timestamp_key}) FROM {target})")
# else:
#     df = spark.sql(f"SELECT * FROM {source}")



In [ ]:
spark.sql("select * from bronze_inventory")